In [1]:
!pip install datasets -q
!pip install hazm --no-deps
!pip install gensim fasttext-wheel flashtext nltk python-crfsuite

**Part one - Preparing and Preprocessing:**

loading dataset:

In [2]:
import json
import requests
from datasets import Dataset, DatasetDict

_URLS = {
    "train": "https://raw.githubusercontent.com/AUT-NLP/PQuAD/main/Dataset/Train.json",
    # "test": "https://raw.githubusercontent.com/AUT-NLP/PQuAD/main/Dataset/Test.json",
}

# Download and parse the data
def load_pquad_data(split):
    """Downloads and parses a PQuAD split."""
    print(f"Downloading {split} data...")
    response = requests.get(_URLS[split])
    response.raise_for_status()

    data = response.json()

    examples = []
    # The structure of the JSON file is { "data": [ { "paragraphs": [ ... ] } ] }
    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qas in paragraph["qas"]:
                question = qas["question"]
                id_ = qas["id"]

                # Extract answers
                answers = qas["answers"]
                answer_starts = [ans["answer_start"] for ans in answers]
                answer_texts = [ans["text"] for ans in answers]

                examples.append({
                    "id": id_,
                    "title": article.get("title", ""),
                    "context": context,
                    "question": question,
                    "answers": {
                        "answer_start": answer_starts,
                        "text": answer_texts,
                    },
                })
    return examples

# Load all splits
train_examples = load_pquad_data("train")
# test_examples = load_pquad_data("test")

# Create Hugging Face Dataset objects
train_dataset = Dataset.from_list(train_examples)
# test_dataset = Dataset.from_list(test_examples)

# Combine them into a single DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    # "test": test_dataset
})

# Explore the loaded data
print("Dataset successfully loaded!")
print("\nDataset information:")
print(dataset)
print("\n" + "="*50 + "\n")

# Check the features (column names and types)
print("Features of the training set:")
print(dataset['train'].features)
print("\n" + "="*50 + "\n")

# Look at the first example in the training set
print("First example in the training set:")
print(dataset['train'][0])

Dataset successfully loaded!

Dataset information:
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 63994
    })
})


Features of the training set:
{'id': Value('string'), 'title': Value('string'), 'context': Value('string'), 'question': Value('string'), 'answers': {'answer_start': List(Value('int64')), 'text': List(Value('string'))}}


First example in the training set:
{'id': '101001', 'title': 'آرسنال', 'context': 'باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Football Club) یک باشگاه فوتبال انگلیسی در شمال شهر لندن است که موفق به کسب ۱۳ عنوان قهرمانی در لیگ دسته اول و لیگ برتر انگلستان، ۱۴ قهرمانی در جام حذفی فوتبال انگلستان ، ۱۶ قهرمانی در جام خیریه انگلستان و دو قهرمانی در جام اتحادیه فوتبال انگلستان شده\u200cاست. آن\u200cها رکورددار طولانی\u200cترین مدت صدرنشینی بدون وقفه در لیگ فوتبال انگلیس، بیشترین بازی بدون باختِ پیاپی (۴۹ بازی) و همچنین قهرمانی بدون شکست در یک فصل (۰۴–۲۰۰۳) می\u200cباشند و توانستند اولین

convert it to corpus and queries:

In [3]:
import pandas as pd
from tqdm.notebook import tqdm

print("Creating the document corpus...")
corpus_data = dataset['train'].select(range(10000))

# We keep the original text for mapping and create a doc_id
corpus_df = pd.DataFrame(corpus_data)['context'].reset_index(drop=True).drop_duplicates().reset_index()
corpus_df.columns = ['doc_id', 'text']
corpus_df['doc_id'] = corpus_df['doc_id'].astype(str)
print(f"Corpus created with {len(corpus_df)} unique documents.")

print("\nCreating the query set...")
queries_data = dataset['train'].select(range(10000))
queries_df = pd.DataFrame(queries_data)[['id', 'question', 'context']]
queries_df.columns = ['query_id', 'question', 'relevant_context']

print(f"Query set created with {len(queries_df)} valid queries.")
print(queries_df.head())

Creating the document corpus...
Corpus created with 1612 unique documents.

Creating the query set...
Query set created with 10000 valid queries.
  query_id                                           question  \
0   101001      موقعیت جغرافی باشگاه فوتبال آرسنال را بگویید؟   
1   101002  لیگ برتر انگلستان موفق به کسب چند عنوان قهرمان...   
2   101003  بیشترین بازی بدون باخت پیاپی متعلق به کدام باش...   
3   101004  باشگاه فوتبال آرسنال موفق به کسب چند عنوان قهر...   
4   101005  باشگاه فوتبال آرسنال چند عنوان قهرمانی در جام ...   

                                    relevant_context  
0  باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...  
1  باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...  
2  باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...  
3  باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...  
4  باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Foo...  


mapping queries to relevant documents in corpus:

In [4]:
print("\nMapping queries to their relevant documents in the corpus...")
# Create a dictionary from original text to doc_id for fast lookup
context_to_doc_id = pd.Series(corpus_df.doc_id.values, index=corpus_df.text).to_dict()
# Map the relevant context from queries to the doc_id in the corpus
queries_df['relevant_doc_id'] = queries_df['relevant_context'].map(context_to_doc_id)

# Drop queries whose relevant document is not in our document corpus
initial_count = len(queries_df)
queries_df.dropna(subset=['relevant_doc_id'], inplace=True)
queries_df['relevant_doc_id'] = queries_df['relevant_doc_id'].astype(str)
final_count = len(queries_df)
print(f"Successfully mapped {final_count}/{initial_count} queries to documents in the corpus.")


Mapping queries to their relevant documents in the corpus...
Successfully mapped 10000/10000 queries to documents in the corpus.


preprocessing and tokenizing:

In [5]:
from hazm import Normalizer, word_tokenize, stopwords_list
import re

# Load hazm tools
normalizer = Normalizer()
persian_stopwords = set(stopwords_list())

def tokenize_text(text):
    """
    A function to normalize, clean, and tokenize Persian text.
    Returns a list of tokens.
    """
    # Normalization
    text = normalizer.normalize(text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords and short words
    tokens = [token for token in tokens if token not in persian_stopwords and len(token) > 1]

    # Return the list of tokens
    return tokens

# Apply tokenization to the documents
print("\nTokenizing documents...")
tqdm.pandas()
corpus_df['tokens'] = corpus_df['text'].progress_apply(tokenize_text)

# Apply tokenization to the questions
print("\nTokenizing questions...")
queries_df['question_tokens'] = queries_df['question'].progress_apply(tokenize_text)

# --- Display Final Results ---
print("\n--- Final DataFrames with Tokens ---")
print("Corpus DataFrame head:")
print(corpus_df[['doc_id', 'tokens']].head())

print("\nQueries DataFrame head:")
print(queries_df[['query_id', 'question_tokens', 'relevant_doc_id']].head())


Tokenizing documents...


  0%|          | 0/1612 [00:00<?, ?it/s]


Tokenizing questions...


  0%|          | 0/10000 [00:00<?, ?it/s]


--- Final DataFrames with Tokens ---
Corpus DataFrame head:
  doc_id                                             tokens
0      0  [باشگاه, فوتبال, ارسنال, انگلیسی, Arsenal, Foo...
1      7  [باشگاه, فوتبال, ارسنال, سال, محله, وولویچ, وا...
2     12  [باشگاه, فوتبال, ارسنال, نام, باشگاه, دیال, اس...
3     18  [سال, هربرت, چاپمن, مربیگری, ارسنال, منصوب, چا...
4     24  [سال, میلادی, برتی, میفیزیوتراپیست, باشگاه, عن...

Queries DataFrame head:
  query_id                                    question_tokens relevant_doc_id
0   101001   [موقعیت, جغرافی, باشگاه, فوتبال, آرسنال, بگویید]               0
1   101002  [لیگ, برتر, انگلستان, موفق, کسب, عنوان, قهرمان...               0
2   101003        [بیشترین, بازی, باخت, پیاپی, متعلق, باشگاه]               0
3   101004  [باشگاه, فوتبال, آرسنال, موفق, کسب, عنوان, قهر...               0
4   101005  [باشگاه, فوتبال, آرسنال, عنوان, قهرمانی, جام, ...               0


**Part two - Init TF-IDF and Retrival:**

init and fit tdidf:

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

all_tokens = corpus_df['tokens'].tolist()

# Initialize TfidfVectorizer
# - tokenizer=lambda x: x tells it to use the pre-tokenized list as is.
# - preprocessor=lambda x: x tells it not to do any default preprocessing.
# - token_pattern=None is important to disable the default token pattern.
vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None
)

# Fit the vectorizer on the combined data to learn the vocabulary
print("\nFitting TfidfVectorizer on the combined vocabulary...")
vectorizer.fit(all_tokens)
print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

# transform the documents and questions into TF-IDF vectors
doc_tfidf_vectors = vectorizer.transform(corpus_df['tokens'])
query_tfidf_vectors = vectorizer.transform(queries_df['question_tokens'])

print("\nShape of document TF-IDF matrix:", doc_tfidf_vectors.shape)
print("Shape of query TF-IDF matrix:", query_tfidf_vectors.shape)


Fitting TfidfVectorizer on the combined vocabulary...
Vocabulary size: 20729

Shape of document TF-IDF matrix: (1612, 20729)
Shape of query TF-IDF matrix: (10000, 20729)


rank documents for each query:

In [7]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

# Create a mapping from the document's integer index in the DataFrame to its string 'doc_id'
index_to_doc_id = corpus_df['doc_id'].tolist()
# Also create the reverse mapping for quick lookups
doc_id_to_index = {doc_id: i for i, doc_id in enumerate(index_to_doc_id)}

print("Calculating cosine similarity for all query-document pairs...")
# This computes the similarity between every query vector and every document vector
# The result is a matrix of shape (num_queries, num_documents)
cosine_sim_matrix = cosine_similarity(query_tfidf_vectors, doc_tfidf_vectors)
print("Similarity matrix calculated.")
print(f"Shape of the similarity matrix: {cosine_sim_matrix.shape}")

# For each query, find the indices of the top 5 most similar documents
# np.argsort sorts in ascending order, so we use [:, ::-1] to get descending order
# Then we select the top 5 indices for each query
print("\nRanking documents for each query...")
top_5_doc_indices = np.argsort(cosine_sim_matrix, axis=1)[:, ::-1][:, :5]
print("Top 5 document indices found for each query.")

Calculating cosine similarity for all query-document pairs...
Similarity matrix calculated.
Shape of the similarity matrix: (10000, 1612)

Ranking documents for each query...
Top 5 document indices found for each query.


**Part three - Measuring System:**

In [8]:
# --- Calculate Evaluation Metrics ---

num_queries = len(queries_df)
precision_at_5_scores = []
recall_at_5_scores = []
reciprocal_ranks = []

# Get the list of correct document IDs for all queries
relevant_doc_ids = queries_df['relevant_doc_id'].tolist()

print("\nCalculating evaluation metrics...")
for i in tqdm(range(num_queries)):
    # Get the correct document ID for the i-th query
    correct_doc_id = relevant_doc_ids[i]

    # Get the indices of the top 5 retrieved documents
    retrieved_indices = top_5_doc_indices[i]

    # Convert the retrieved indices back to their document IDs
    retrieved_doc_ids = [index_to_doc_id[idx] for idx in retrieved_indices]

    # --- Precision@5 and Recall@5 ---
    is_relevant_in_top_5 = 1 if correct_doc_id in retrieved_doc_ids else 0

    precision_at_5 = is_relevant_in_top_5 / 5
    recall_at_5 = is_relevant_in_top_5 / 1

    precision_at_5_scores.append(precision_at_5)
    recall_at_5_scores.append(recall_at_5)

    # --- Mean Reciprocal Rank (MRR) ---
    # Find the rank of the first correct document in the full sorted list
    all_sorted_indices = np.argsort(cosine_sim_matrix[i], axis=0)[::-1]

    # Find the 0-based rank of the correct document's index
    correct_doc_index = doc_id_to_index[correct_doc_id]
    rank = np.where(all_sorted_indices == correct_doc_index)[0][0]
    reciprocal_rank = 1 / (rank + 1)

    reciprocal_ranks.append(reciprocal_rank)

# --- Report the Final Results ---

avg_precision_at_5 = np.mean(precision_at_5_scores)
avg_recall_at_5 = np.mean(recall_at_5_scores)
mean_reciprocal_rank = np.mean(reciprocal_ranks)

print("\n" + "="*50)
print("           EVALUATION RESULTS")
print("="*50)
print(f"Average Precision@5:  {avg_precision_at_5:.4f}")
print(f"Average Recall@5:     {avg_recall_at_5:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mean_reciprocal_rank:.4f}")
print("="*50)


Calculating evaluation metrics...


  0%|          | 0/10000 [00:00<?, ?it/s]


           EVALUATION RESULTS
Average Precision@5:  0.1682
Average Recall@5:     0.8409
Mean Reciprocal Rank (MRR): 0.7117


**show worst perfomance queries:**

In [9]:
# Find the indices of the 5 queries with the lowest reciprocal rank (worst performance)
# np.argsort sorts in ascending order, so the first 5 are the worst
worst_query_indices = np.argsort(reciprocal_ranks)[:5]

print("="*80)
print("           ANALYSIS OF THE 5 WORST PERFORMING QUERIES")
print("="*80)

# Loop through each of the worst-performing queries
for i, query_idx in enumerate(worst_query_indices):

    # Get Question and Performance Details ---
    question_text = queries_df.iloc[query_idx]['question']
    correct_doc_id = queries_df.iloc[query_idx]['relevant_doc_id']
    reciprocal_rank = reciprocal_ranks[query_idx]

    # Find the actual rank (1-based) of the correct document
    sorted_doc_indices = np.argsort(cosine_sim_matrix[query_idx], axis=0)[::-1]
    correct_doc_index = doc_id_to_index[correct_doc_id]
    rank_of_correct_doc = np.where(sorted_doc_indices == correct_doc_index)[0][0] + 1

    print(f"\n--- Analyzing the #{i+1} Worst Performing Query ---")
    print(f"**Question:** {question_text}")
    print(f"**Mean Reciprocal Rank:** {reciprocal_rank:.4f}")
    print(f"**Rank of Correct Document:** {rank_of_correct_doc} out of {len(corpus_df)}")
    print("-" * 50)

    # --- Show the Correct Document ---
    correct_doc_text = corpus_df[corpus_df['doc_id'] == correct_doc_id]['text'].values[0]
    print(f"**Correct Document (ID: {correct_doc_id}):**")
    print(f"{correct_doc_text}")
    print("-" * 50)

    # --- Show the Top 3 Documents Ordered by the System ---
    print("**System's Top 3 Retrieved Documents:**")

    # Get the top 3 document indices for this query
    top_3_indices = sorted_doc_indices[:3]

    for rank, doc_idx in enumerate(top_3_indices):
        retrieved_doc_id = index_to_doc_id[doc_idx]
        similarity_score = cosine_sim_matrix[query_idx, doc_idx]
        retrieved_doc_text = corpus_df.iloc[doc_idx]['text']

        is_correct = "✅ (Correct Document)" if retrieved_doc_id == correct_doc_id else ""

        print(f"\n  Rank {rank + 1}:")
        print(f"    - Score: {similarity_score:.4f}")
        print(f"    - Doc ID: {retrieved_doc_id} {is_correct}")
        print(f"    - Text: {retrieved_doc_text}")

    print("\n" + "="*80)


           ANALYSIS OF THE 5 WORST PERFORMING QUERIES

--- Analyzing the #1 Worst Performing Query ---
**Question:** رندر چیست؟
**Mean Reciprocal Rank:** 0.0006
**Rank of Correct Document:** 1601 out of 1612
--------------------------------------------------
**Correct Document (ID: 6877):**
بالا بردن میزان نرخ زمانی کلاک و که سبب تولید هش‌ریت (میزان محاسبه اطلاعات توسط پردازنده در واحد زمان است)، بیشتر در واحد زمان و انجام سریع تری محاسبه در بازه زمانی می‌شود. پردازنده‌هایی که قفلشان بازگشایی شده باشد را می‌توان اورکلاک کرد و مزیت اورکلاک، انجام سریع تر پردازش و رندر‌های سنگین توسط پردازنده است. مضرات اورکلاک بالا رفتن دمای پردازنده برای محاسبه و در نتیجه استفاده مداوم سبب پایین آمدن عمر پردازش گر می‌شود. عمل Overclocking نیازمند دانش کافی در زمینه سخت افزار می‌باشد و هرگونه اقدام نادرست، آسیب‌های جبران ناپذیری به پردازنده ها وارد می‌کند. تنها پردازنده‌هایی اورکلاک می‌شوند که قابلیت اورکلاک شدن را در پسوند خود داشته باشند.
--------------------------------------------------
**System's

**Repeat with no PreProcessing and just Tokenizing:**

In [10]:
def tokenize_text_simple(text):
    """
    A function that ONLY tokenizes the text.
    It performs no normalization, stopword removal, or cleaning.
    """
    return word_tokenize(text)

# Apply simple tokenization to the documents
print("\nTokenizing documents (no preprocessing)...")
tqdm.pandas()
corpus_df['tokens'] = corpus_df['text'].progress_apply(tokenize_text_simple)

# Apply simple tokenization to the questions
print("\nTokenizing questions (no preprocessing)...")
queries_df['question_tokens'] = queries_df['question'].progress_apply(tokenize_text_simple)

print("\n--- Sample of Simple Tokenization ---")
print("Original Text:\n", corpus_df['text'].iloc[0])
print("\nTokens:\n", corpus_df['tokens'].iloc[0])
print("-" * 30)



Tokenizing documents (no preprocessing)...


  0%|          | 0/1612 [00:00<?, ?it/s]


Tokenizing questions (no preprocessing)...


  0%|          | 0/10000 [00:00<?, ?it/s]


--- Sample of Simple Tokenization ---
Original Text:
 باشگاه فوتبال آرسنال (به انگلیسی: Arsenal Football Club) یک باشگاه فوتبال انگلیسی در شمال شهر لندن است که موفق به کسب ۱۳ عنوان قهرمانی در لیگ دسته اول و لیگ برتر انگلستان، ۱۴ قهرمانی در جام حذفی فوتبال انگلستان ، ۱۶ قهرمانی در جام خیریه انگلستان و دو قهرمانی در جام اتحادیه فوتبال انگلستان شده‌است. آن‌ها رکورددار طولانی‌ترین مدت صدرنشینی بدون وقفه در لیگ فوتبال انگلیس، بیشترین بازی بدون باختِ پیاپی (۴۹ بازی) و همچنین قهرمانی بدون شکست در یک فصل (۰۴–۲۰۰۳) می‌باشند و توانستند اولین و تنها تیمی در تاریخ لیگ برتر باشند که جام طلایی را بدست می‌آورند.

Tokens:
 ['باشگاه', 'فوتبال', 'آرسنال', '(', 'به', 'انگلیسی', ':', 'Arsenal', 'Football', 'Club', ')', 'یک', 'باشگاه', 'فوتبال', 'انگلیسی', 'در', 'شمال', 'شهر', 'لندن', 'است', 'که', 'موفق', 'به', 'کسب', '۱۳', 'عنوان', 'قهرمانی', 'در', 'لیگ', 'دسته', 'اول', 'و', 'لیگ', 'برتر', 'انگلستان', '،', '۱۴', 'قهرمانی', 'در', 'جام', 'حذفی', 'فوتبال', 'انگلستان', '،', '۱۶', 'قهرمانی', 'در', 'جام', 

In [11]:
# Combine all tokens to build a common vocabulary
all_tokens = corpus_df['tokens'].tolist() + queries_df['question_tokens'].tolist()

vectorizer = TfidfVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None
)

print("\nFitting TfidfVectorizer on the combined vocabulary...")
vectorizer.fit(all_tokens)
print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")

doc_tfidf_vectors = vectorizer.transform(corpus_df['tokens'])
query_tfidf_vectors = vectorizer.transform(queries_df['question_tokens'])

print("\nShape of document TF-IDF matrix:", doc_tfidf_vectors.shape)
print("Shape of query TF-IDF matrix:", query_tfidf_vectors.shape)



Fitting TfidfVectorizer on the combined vocabulary...
Vocabulary size: 24505

Shape of document TF-IDF matrix: (1612, 24505)
Shape of query TF-IDF matrix: (10000, 24505)


In [12]:
index_to_doc_id = corpus_df['doc_id'].tolist()
doc_id_to_index = {doc_id: i for i, doc_id in enumerate(index_to_doc_id)}

print("\nCalculating cosine similarity for all query-document pairs...")
cosine_sim_matrix = cosine_similarity(query_tfidf_vectors, doc_tfidf_vectors)
print("Similarity matrix calculated.")

print("\nRanking documents for each query...")
top_5_doc_indices = np.argsort(cosine_sim_matrix, axis=1)[:, ::-1][:, :5]
print("Top 5 document indices found for each query.")


Calculating cosine similarity for all query-document pairs...
Similarity matrix calculated.

Ranking documents for each query...
Top 5 document indices found for each query.


In [13]:
num_queries = len(queries_df)
precision_at_5_scores = []
recall_at_5_scores = []
reciprocal_ranks = []
relevant_doc_ids = queries_df['relevant_doc_id'].tolist()

print("\nCalculating evaluation metrics...")
for i in tqdm(range(num_queries)):
    correct_doc_id = relevant_doc_ids[i]
    retrieved_indices = top_5_doc_indices[i]
    retrieved_doc_ids = [index_to_doc_id[idx] for idx in retrieved_indices]

    # Precision@5 and Recall@5
    is_relevant_in_top_5 = 1 if correct_doc_id in retrieved_doc_ids else 0
    precision_at_5 = is_relevant_in_top_5 / 5
    recall_at_5 = is_relevant_in_top_5 / 1

    precision_at_5_scores.append(precision_at_5)
    recall_at_5_scores.append(recall_at_5)

    # MRR
    all_sorted_indices = np.argsort(cosine_sim_matrix[i], axis=0)[::-1]
    try:
        correct_doc_index = doc_id_to_index[correct_doc_id]
        rank = np.where(all_sorted_indices == correct_doc_index)[0][0]
        reciprocal_rank = 1 / (rank + 1)
    except KeyError:
        reciprocal_rank = 0
    reciprocal_ranks.append(reciprocal_rank)

# ---  Report the Final Results ---

avg_precision_at_5 = np.mean(precision_at_5_scores)
avg_recall_at_5 = np.mean(recall_at_5_scores)
mean_reciprocal_rank = np.mean(reciprocal_ranks)

print("\n" + "="*50)
print("           EVALUATION RESULTS (NO PREPROCESSING)")
print("="*50)
print(f"Average Precision@5:  {avg_precision_at_5:.4f}")
print(f"Average Recall@5:     {avg_recall_at_5:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mean_reciprocal_rank:.4f}")
print("="*50)


Calculating evaluation metrics...


  0%|          | 0/10000 [00:00<?, ?it/s]


           EVALUATION RESULTS (NO PREPROCESSING)
Average Precision@5:  0.1676
Average Recall@5:     0.8379
Mean Reciprocal Rank (MRR): 0.7236


In [14]:
# Find the indices of the 5 queries with the lowest reciprocal rank (worst performance)
# np.argsort sorts in ascending order, so the first 5 are the worst
worst_query_indices = np.argsort(reciprocal_ranks)[:5]

print("="*80)
print("           ANALYSIS OF THE 5 WORST PERFORMING QUERIES")
print("="*80)

# Loop through each of the worst-performing queries
for i, query_idx in enumerate(worst_query_indices):

    # --- Get Question and Performance Details ---
    question_text = queries_df.iloc[query_idx]['question']
    correct_doc_id = queries_df.iloc[query_idx]['relevant_doc_id']
    reciprocal_rank = reciprocal_ranks[query_idx]

    # Find the actual rank (1-based) of the correct document
    sorted_doc_indices = np.argsort(cosine_sim_matrix[query_idx], axis=0)[::-1]
    correct_doc_index = doc_id_to_index[correct_doc_id]
    rank_of_correct_doc = np.where(sorted_doc_indices == correct_doc_index)[0][0] + 1

    print(f"\n--- Analyzing the #{i+1} Worst Performing Query ---")
    print(f"**Question:** {question_text}")
    print(f"**Mean Reciprocal Rank:** {reciprocal_rank:.4f}")
    print(f"**Rank of Correct Document:** {rank_of_correct_doc} out of {len(corpus_df)}")
    print("-" * 50)

    # --- Show the Correct Document ---
    correct_doc_text = corpus_df[corpus_df['doc_id'] == correct_doc_id]['text'].values[0]
    print(f"**Correct Document (ID: {correct_doc_id}):**")
    print(f"{correct_doc_text}")
    print("-" * 50)

    # --- Show the Top 3 Documents Ordered by the System ---
    print("**System's Top 3 Retrieved Documents:**")

    # Get the top 3 document indices for this query
    top_3_indices = sorted_doc_indices[:3]

    for rank, doc_idx in enumerate(top_3_indices):
        retrieved_doc_id = index_to_doc_id[doc_idx]
        similarity_score = cosine_sim_matrix[query_idx, doc_idx]
        retrieved_doc_text = corpus_df.iloc[doc_idx]['text']

        is_correct = "✅ (Correct Document)" if retrieved_doc_id == correct_doc_id else ""

        print(f"\n  Rank {rank + 1}:")
        print(f"    - Score: {similarity_score:.4f}")
        print(f"    - Doc ID: {retrieved_doc_id} {is_correct}")
        print(f"    - Text: {retrieved_doc_text}")

    print("\n" + "="*80)


           ANALYSIS OF THE 5 WORST PERFORMING QUERIES

--- Analyzing the #1 Worst Performing Query ---
**Question:** رندر چیست؟
**Mean Reciprocal Rank:** 0.0006
**Rank of Correct Document:** 1606 out of 1612
--------------------------------------------------
**Correct Document (ID: 6877):**
بالا بردن میزان نرخ زمانی کلاک و که سبب تولید هش‌ریت (میزان محاسبه اطلاعات توسط پردازنده در واحد زمان است)، بیشتر در واحد زمان و انجام سریع تری محاسبه در بازه زمانی می‌شود. پردازنده‌هایی که قفلشان بازگشایی شده باشد را می‌توان اورکلاک کرد و مزیت اورکلاک، انجام سریع تر پردازش و رندر‌های سنگین توسط پردازنده است. مضرات اورکلاک بالا رفتن دمای پردازنده برای محاسبه و در نتیجه استفاده مداوم سبب پایین آمدن عمر پردازش گر می‌شود. عمل Overclocking نیازمند دانش کافی در زمینه سخت افزار می‌باشد و هرگونه اقدام نادرست، آسیب‌های جبران ناپذیری به پردازنده ها وارد می‌کند. تنها پردازنده‌هایی اورکلاک می‌شوند که قابلیت اورکلاک شدن را در پسوند خود داشته باشند.
--------------------------------------------------
**System's